In [9]:

import pandas as pd
import numpy as np
import bz2
import json
import requests
import findspark
findspark.init('/Users/tatianacogne/spark')
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from datetime import datetime
import sys
import urllib.request as r
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
from urllib.request import urlopen, Request
import csv

In [10]:
# Create a spark context
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
# 2020 
df = spark.read.json('data/quotes-2020.json.bz2')

### Process df with pypsark
Note: not able to put it back in a JSON file. 

In [ ]:
df = df.select([c for c in df.columns if c in ['date','qids','quotation','quoteID']]).filter(sf.size('qids') > 0)
rdd2=df.rdd.map(lambda x: (datetime.strptime(x['date'], '%Y-%m-%d %H:%M:%S'),x['quoteID'][11:],x['quotation'],x['qids']))  
df2=rdd2.toDF(['date','quoteID','quotation','qids'])

### 2020

In [ ]:
path_to_file = 'data/quotes-2020.json.bz2' 
path_to_out = 'data/sp_qids_quotes-2020-process.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            #instance['date'] = datetime.strptime(instance['date'], '%Y-%m-%d %H:%M:%S') 
            instance['quoteID']= instance['quoteID'][11:]
            if(len(instance['qids'])!=0):
                instance['qids'] = instance['qids'][0]
                instance = dict((key,value) for key, value in instance.items() if key in('date','quoteID','quotation','qids'))
                d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

### 2019

In [ ]:
path_to_file = 'data/quotes-2019.json.bz2' 
path_to_out = 'data/quotes-2019-process.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            #instance['date'] = datetime.strptime(instance['date'], '%Y-%m-%d %H:%M:%S') 
            instance['quoteID']= instance['quoteID'][11:]
            if(len(instance['qids'])!=0):
                instance['qids'] = instance['qids'][0]
                instance = dict((key,value) for key, value in instance.items() if key in('date','quoteID','quotation','qids'))
                d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

### 2018

In [ ]:
path_to_file = 'data/quotes-2018.json.bz2' 
path_to_out = 'data/quotes-2018-process.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            #instance['date'] = datetime.strptime(instance['date'], '%Y-%m-%d %H:%M:%S') 
            instance['quoteID']= instance['quoteID'][11:]
            if(len(instance['qids'])!=0):
                instance['qids'] = instance['qids'][0]
                instance = dict((key,value) for key, value in instance.items() if key in('date','quoteID','quotation','qids'))
                d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

### 2017

In [ ]:
path_to_file = 'data/quotes-2017.json.bz2' 
path_to_out = 'data/quotes-2017-process.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            #instance['date'] = datetime.strptime(instance['date'], '%Y-%m-%d %H:%M:%S') 
            instance['quoteID']= instance['quoteID'][11:]
            if(len(instance['qids'])!=0):
                instance['qids'] = instance['qids'][0]
                instance = dict((key,value) for key, value in instance.items() if key in('date','quoteID','quotation','qids'))
                d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

### 2016

In [ ]:
path_to_file = 'data/quotes-2016.json.bz2' 
path_to_out = 'data/quotes-2016-process.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            #instance['date'] = datetime.strptime(instance['date'], '%Y-%m-%d %H:%M:%S') 
            instance['quoteID']= instance['quoteID'][11:]
            if(len(instance['qids'])!=0):
                instance['qids'] = instance['qids'][0]
                instance = dict((key,value) for key, value in instance.items() if key in('date','quoteID','quotation','qids'))
                d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

### 2015

In [ ]:
path_to_file = 'data/quotes-2015.json.bz2' 
path_to_out = 'data/quotes-2015-process.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            #instance['date'] = datetime.strptime(instance['date'], '%Y-%m-%d %H:%M:%S') 
            instance['quoteID']= instance['quoteID'][11:]
            if(len(instance['qids'])!=0):
                instance['qids'] = instance['qids'][0]
                instance = dict((key,value) for key, value in instance.items() if key in('date','quoteID','quotation','qids'))
                d_file.write((json.dumps(instance)+'\n').encode('utf-8'))

### Speakers

In [11]:
df_sp = pd.read_parquet('data/speaker_attributes.parquet')

In [12]:
df_sp = df_sp[['id','label','nationality','occupation','ethnic_group', 'party','religion']].copy()
df_sp = df_sp.rename(columns={"nationality": "nationality_qids", "ethnic_group": "ethnic_qids","occupation":"occupation_qids","party":'party_qids','id':'speaker_qids','label':'speaker_label','religion':'religion_qids'})

#### QIDS of all different nationalities, occupations, ethnic, party, religion 

In [13]:
nationality_qids = df_sp['nationality_qids'].explode('nationality_qids').drop_duplicates().values
occupation_qids = df_sp['occupation_qids'].explode('occupation_qids').drop_duplicates().values
ethnic_qids = df_sp['ethnic_qids'].explode('ethnic_qids').drop_duplicates().values
party_qids = df_sp['party_qids'].explode('party_qids').drop_duplicates().values
religion_qids = df_sp['religion_qids'].explode('religion_qids').drop_duplicates()

In [31]:
def find_label(qids):
    url = 'https://www.wikidata.org/wiki/'
    url_qids = url+qids
    print(qids)
    ru =r.Request(url_qids,headers={'Connection': 'close'})
    page = urlopen(ru).read()
    soup = BeautifulSoup(page)
    label = soup.find("span", {"class":"wikibase-title-label"}).text
    return label 

#### Religion QIDS - label

In [ ]:
religion_qids = [x for x in religion_qids if x!=None]
df_religion = pd.DataFrame()
df_religion['qids'] = religion_qids
df_religion['label'] = df_religion['qids'].apply(lambda x : find_label(x))

In [ ]:
df_religion.to_json('data/religion.json')

#### Nationality QIDS - label

In [ ]:
nationality_qids = [x for x in nationality_qids if x!=None]
df_nationality = pd.DataFrame()
df_nationality['qids'] = nationality_qids
df_nationality['label'] = df_nationality['qids'].apply(lambda x : find_label(x))

In [ ]:
df_nationality.to_json('data/nationality.json')

#### Ethnic QIDS - label

In [ ]:
ethnic_qids = [x for x in ethnic_qids if x!=None]
df_ethnic = pd.DataFrame()
df_ethnic['ethnic_qids'] = ethnic_qids
df_ethnic['ethnic_label'] = df_ethnic['ethnic_qids'].apply(lambda x : find_label(x))

In [ ]:
df_ethnic.to_json('data/ethnic.json')

#### Party QIDS - label 
- n° of party: 9'632
- Problem: Q99761286 - <Response [404]>

In [ ]:
party_qids = [x for x in party_qids if x!=None]
df_party = pd.DataFrame()
df_party['party_qids'] = party_qids
df_party = df_party[df_party.party_qids != 'Q99761286']
df_party['party_label'] = df_party['party_qids'].apply(lambda x : find_label(x))

In [ ]:
# Code to find the Respons [404]
#df_party['party_url'] = df_party['party_qids'].apply(lambda x :'https://www.wikidata.org/wiki/'+x)
#df_party['response'] = df_party['party_url'].apply(lambda x : requests.get(x))
#df_party['response'] = df_party['party_url'].apply(lambda x : requests.get(x))

In [ ]:
df_party.to_json('data/party.json')

#### Occupation QIDS - label
- Need to split in 3 arrays in order to be more efficient
- 'Q57557390', 'Q102046591', 'Q98384826', 'Q105645755', 'Q99753484','Q56411328', 'Q96143085', 'Q96144081', 'Q3738699', 'Q5568256'

In [35]:
occupation_qids = [x for x in occupation_qids if x!=None]
l = len(occupation_qids)
l1 = 500
l2 = 1000
occupation_qids_1 = occupation_qids[:l1]
occupation_qids_2 = occupation_qids[l1:l2]
occupation_qids_3 = occupation_qids[1000:5000]
occupation_qids_4 = occupation_qids[5000:10000]
occupation_qids_5 = occupation_qids[10000:]
a = ['Q57557390', 'Q102046591', 'Q98384826', 'Q105645755', 'Q99753484','Q56411328','Q96143085', 'Q96144081','Q3738699','Q5568256']
occupation_qids_3 = [x for x in occupation_qids_3 if x not in a ]
occupation_qids_4 = [x for x in occupation_qids_4 if x not in a ]
occupation_qids_5 = [x for x in occupation_qids_5 if x not in a ]

In [36]:
df_occupation = pd.DataFrame()
df_occupation['occupation_qids'] = occupation_qids_5
#df_occupation['occupation_url'] = df_occupation['occupation_qids'].apply(lambda x :'https://www.wikidata.org/wiki/'+x)
#df_occupation['response'] = df_occupation['occupation_url'].apply(lambda x : requests.get(x))
#df_occupation['response_text'] = df_occupation['response'].apply(lambda x : str(x))
df_occupation['occupation_label'] = df_occupation['occupation_qids'].apply(lambda x : find_label(x))

Q22815070
Q3753649
Q1438738
Q510865
Q3606416
Q15992673
Q4426313
Q272826
Q37260
Q2048676
Q4171318
Q1262105
Q4412544
Q15271009
Q1179368
Q1723169
Q1139358
Q842803
Q98273907
Q1246370
Q2071566
Q138365
Q27443
Q1143617
Q30119825
Q1081491
Q17638669
Q29652773
Q6050670
Q635713
Q1422577
Q1423756
Q1237928
Q55400155
Q4287237
Q55631051
Q11848548
Q28922814
Q104816290
Q54627009
Q47517033
Q51099445
Q66604809
Q1208997
Q1470970
Q7208069
Q104707848
Q11204
Q634995
Q7892548
Q377688
Q3326688
Q5148404
Q7409248
Q25535537
Q63031501
Q884053
Q498640
Q843281
Q19004821
Q6434389
Q168796
Q1191714
Q15722942
Q877955
Q3778055
Q30332887
Q105670486
Q63439712
Q2361374
Q40182607
Q2868277
Q16533346
Q52145133
Q47004158
Q5615055
Q1787885
Q4479395
Q4354908
Q50418136
Q33506
Q5719186
Q60649184
Q51079076
Q51228702
Q58333
Q80895
Q12198
Q50410668
Q9009117
Q48956331
Q48956358
Q59057
Q78952848
Q11504425
Q281111
Q3311406
Q3687702
Q637324
Q3824194
Q6038246
Q1309993
Q1006696
Q14623843
Q6662173
Q2754873
Q4741053
Q10669380
Q55383242
Q55670

,occupation_qids,occupation_label
0,Q3140857,horticulturist
1,Q7301138,real estate entrepreneur
2,Q26224000,castellologist
3,Q81710,shepherd
4,Q17504942,scuba diver
...,...,...
3994,Q30132173,angiologist
3995,Q7108,biotechnology
3996,Q1781945,Vardapet
3997,Q187021,fiscal policy


In [ ]:
df_occupation.to_json('data/occupation5.json')

In [ ]:
df1 = pd.read_json('data/occupation1.json')
df2 = pd.read_json('data/occupation2.json')
df3 = pd.read_json('data/occupation3.json')
df4 = pd.read_json('data/occupation4.json')
df5 = pd.read_json('data/occupation5.json')

In [ ]:
df = pd.concat([df1,df2,df3])

In [ ]:
df_occupation['response_text']